In [1]:
!pip install -q langchain-openai langchain-core requests

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


Tool Creation

In [10]:

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

print(multiply.invoke({'a':3, 'b':4}))
print(multiply.name)
print(multiply.description)
print(multiply.args)

12
multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


Tool Binding

In [11]:
model.invoke('Hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--46844c13-4f3d-4f91-85d5-6ffcf7a252be-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [12]:
llm_with_tool = model.bind_tools([multiply])

In [15]:
llm_with_tool.invoke('Hi')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--3647467a-20ff-4671-9359-aa689dad2bd9-0', usage_metadata={'input_tokens': 21, 'output_tokens': 10, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})

In [57]:
query = HumanMessage('can you multiply 3 with 1000')

In [58]:
message = [query]
message

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

Tool Execution

In [59]:
# result = llm_with_tool.invoke('can you multiply 3 with 10')
result = llm_with_tool.invoke(message)

In [60]:
message.append(result)

In [61]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'b': 1000.0, 'a': 3.0},
 'id': '4026d5c0-3a6c-4edd-8ba7-73360b997e9d',
 'type': 'tool_call'}

In [62]:
tool_result = multiply.invoke(result.tool_calls[0])

In [63]:
message

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 1000.0, "a": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--9803bda4-8a8c-4d6c-9450-305a787f9b3d-0', tool_calls=[{'name': 'multiply', 'args': {'b': 1000.0, 'a': 3.0}, 'id': '4026d5c0-3a6c-4edd-8ba7-73360b997e9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [64]:
message.append(tool_result)

In [65]:
message

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 1000.0, "a": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--9803bda4-8a8c-4d6c-9450-305a787f9b3d-0', tool_calls=[{'name': 'multiply', 'args': {'b': 1000.0, 'a': 3.0}, 'id': '4026d5c0-3a6c-4edd-8ba7-73360b997e9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='4026d5c0-3a6c-4edd-8ba7-73360b997e9d')]

In [66]:
llm_with_tool.invoke(message).content

'3 multiplied by 1000 is 3000.'